In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import torch

# โหลด tokenizer และ model
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")





/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [20]:
import pandas as pd

# โหลดข้อมูลจากไฟล์ CSV
file_path = "/content/mdtesting.csv"  # เปลี่ยนเส้นทางไฟล์ตามความต้องการ
df = pd.read_csv(file_path)
texts = df["text"].tolist()  # สมมติว่าไฟล์มีคอลัมน์ชื่อ "text"


In [21]:
def create_prompt(text):
    prompt = (
        f"{text}\n\n"
        "Identify and output as [[<symbol>SYMBOL<aspect>ASPECT<opinion>OPINION<sentiment>SENTIMENT]]."
    )
    return prompt


In [22]:
# เตรียมข้อมูล input
inputs = []
for text in texts:
    prompt = create_prompt(text)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    inputs.append(input_ids)

# สร้างผลลัพธ์หลายชุด
outputs = []
for input_ids in inputs:
    output_ids = model.generate(
        input_ids,
        max_length=128,
        num_beams=5,
        num_return_sequences=5,
        early_stopping=True
    )
    outputs.append(output_ids)

In [23]:
# แปลงผลลัพธ์ที่ได้
results = []
for output in outputs:
    decoded_texts = []
    for i in range(5):  # ผลลัพธ์ที่เป็นหลายชุด
        decoded_text = tokenizer.decode(output[i], skip_special_tokens=True)

        # เปลี่ยน <extra_id_0> และ token ต่าง ๆ เป็นรูปแบบที่ต้องการ
        decoded_text = decoded_text.replace("<extra_id_0>", "<SYMBOL>")
        decoded_text = decoded_text.replace("<extra_id_1>", "<ASPECT>")
        decoded_text = decoded_text.replace("<extra_id_2>", "<OPINION>")
        decoded_text = decoded_text.replace("<extra_id_3>", "<SENTIMENT>")

        decoded_texts.append(decoded_text)
    results.append(decoded_texts)


In [24]:
# แสดงผลลัพธ์ที่ได้
for i, text in enumerate(texts):
    print(f"Input: {text}")
    for j, result in enumerate(results[i]):
        print(f"GenResult-{j+1}: {result}")
    print("\n" + "-"*50 + "\n")

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
GenResult-4: <SYMBOL> BJC ร่วง 17%
GenResult-5: <SYMBOL> (<symbol>)

--------------------------------------------------

Input: STA พ้นจุดต่ำสุด! ลุ้นรายได้ Q4 ฟื้น รับส่งออก “ยาง” เพิ่ม-ราคาขายสูง
GenResult-1: <SYMBOL> “ยาง” เพิ่ม-ราคาขาย
GenResult-2: <SYMBOL> “ยาง” พ้นจุดต่ํา
GenResult-3: <SYMBOL> “ยาง” พ้นจุด...
GenResult-4: <SYMBOL> “ยาง” หลุดจุดต่ํา
GenResult-5: <SYMBOL> “ยาง” พ้นจุด

--------------------------------------------------

Input: STA พ้นจุดต่ำสุด! ลุ้นรายได้ Q4 ฟื้น รับส่งออก “ยาง” เพิ่ม-ราคาขายสูง
GenResult-1: <SYMBOL> “ยาง” เพิ่ม-ราคาขาย
GenResult-2: <SYMBOL> “ยาง” พ้นจุดต่ํา
GenResult-3: <SYMBOL> “ยาง” พ้นจุด...
GenResult-4: <SYMBOL> “ยาง” หลุดจุดต่ํา
GenResult-5: <SYMBOL> “ยาง” พ้นจุด

--------------------------------------------------

Input: STA พ้นจุดต่ำสุด! ลุ้นรายได้ Q4 ฟื้น รับส่งออก “ยาง” เพิ่ม-ราคาขายสูง
GenResult-1: <SYMBOL> “ยาง” เพิ่ม-ราคาขาย
GenResult-2: <SYMBOL> “ยาง” พ้นจุดต่ํา
GenResult-3: <SY

In [25]:
#  จัดข้อมูลเพื่อสร้าง DataFrame โดยรวม texts และ results เข้าด้วยกัน
data = []
for i, text in enumerate(texts):
    row = {
        "text": text,
        "GenResult-1": results[i][0],
        "GenResult-2": results[i][1],
        "GenResult-3": results[i][2],
        "GenResult-4": results[i][3],
        "GenResult-5": results[i][4]
    }
    data.append(row)

# สร้าง DataFrame จากข้อมูลที่เตรียมไว้
output_df = pd.DataFrame(data)

# บันทึก DataFrame เป็นไฟล์ CSV
output_df.to_csv("gen_results.csv", index=False, encoding="utf-8-sig")

print("ไฟล์ CSV ถูกบันทึกแล้วในชื่อ gen_results.csv")

ไฟล์ CSV ถูกบันทึกแล้วในชื่อ gen_results.csv
